In [12]:
import joblib
from rdkit.Chem.Draw import IPythonConsole #Needed to show molecules+
from rdkit.Chem import Draw
from rdkit.Chem import AllChem as Chem
import gzip
import os
import sys
import csv
import pandas as pd

from syba.syba import SybaClassifier, SmiMolSupplier
import statsmodels.api as sm
from datetime import datetime
import tensorflow as tf

from tensorflow import keras
import numpy as np

from tensorflow.keras import optimizers

from tensorflow.keras import layers
from tensorflow.keras import models

In [2]:
def create_my_model():
    model = keras.Sequential()
    model.add(layers.Embedding(input_dim=1024, output_dim=10))
    model.add(layers.LSTM(128))
    model.add(layers.Dense(128))
    model.add(layers.Dense(64))
    model.add(layers.Dense(32))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['mae', 'accuracy'])
    return model

In [3]:
def create_my_model10():
    model = keras.Sequential()
    model.add(layers.Embedding(input_dim=1024, output_dim=100))
    model.add(layers.LSTM(264))
    model.add(layers.Dense(264))
    model.add(layers.Dense(128))
    model.add(layers.Dense(64))
    model.add(layers.Dense(32))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['mae', 'accuracy'])
    return model

In [4]:
rnn_load = create_my_model10()
rnn_load.load_weights('./rnn_lstm_V10.1000.10_weights.h5')
rnn_load.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         102400    
_________________________________________________________________
lstm (LSTM)                  (None, 264)               385440    
_________________________________________________________________
dense (Dense)                (None, 264)               69960     
_________________________________________________________________
dense_1 (Dense)              (None, 128)               33920     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 3

# Comparer a l'Accuracy de SYBA sur un autre fichier de test

In [5]:
#D'abord charger syba et les données de test
syba = SybaClassifier()Mazigh0

syba.fitDefaultScore()


In [18]:
nBits = 1024
inpath = "../data/transfer/Banques/"
outpath = "../out/RNN/V10/"
files = ["Alcools_difficiles.csv", "Alcools_faciles.csv", "Amides_difficiles_PubChem.csv", "Esters_difficiles_PubChem.csv", "Esters_faciles_PubChem.csv", "Imines_difficiles_PubChem.csv", "Imines_faciles_PubChem.csv"]
for f in files:
    with open(inpath+f) as inp, open(outpath+f, "w") as out:
        header = inp.readline().strip()
        print("Processing the files ", f)
        out.write(header)
        out.write("smiles,SybaScore,LstmScore\n")
        data = np.array([np.array(Chem.GetMorganFingerprintAsBitVect(spls[0],2,nBits=nBits)) for spls in SmiMolSupplier(open(inpath+f, mode="rt"), header=True, smi_col=0)])    
        pr = rnn_load.predict(data)
        i = 0
        for line in inp:
            smi = line.strip().split(",")
            out.write("{},{},{}\n".format(smi, syba.predict(line), pr[i]))
            i += 1

Processing the files  Alcools_difficiles.csv
Processing the files  Alcools_faciles.csv
Processing the files  Amides_difficiles_PubChem.csv
Processing the files  Esters_difficiles_PubChem.csv
Processing the files  Esters_faciles_PubChem.csv
Processing the files  Imines_difficiles_PubChem.csv
Processing the files  Imines_faciles_PubChem.csv
